In [7]:
#impot dependencies
import pandas as pd
import numpy as np
import ast 

In [3]:
all_organizations = pd.read_csv("All_Organizations.csv")
all_organizations_df = pd.DataFrame(all_organizations)
all_organizations_df.head()

,Unnamed: 0,advisories,category,cause,charityName,charityNavigatorURL,currentRating,donationAddress,ein,irsClassification,mailingAddress,mission,orgID,organization,tagLine,websiteURL
0,0,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Research and Public Policy',...","{'causeID': 35, 'causeName': 'Non-Medical Scie...",Mount Desert Island Biological Laboratory,https://www.charitynavigator.org/?bay=search.s...,"{'score': 92.61, 'ratingID': 131800, 'publicat...","{'country': None, 'stateOrProvince': 'ME', 'ci...",10202467,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'ME', 'ci...",The MDI Biological Laboratory is a rapidly gro...,5954,{'charityName': 'Mount Desert Island Biologica...,"Connecting Science, Environment, and Health",http://www.mdibl.org/
1,1,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Community Development', 'cat...","{'causeID': 42, 'causeName': 'United Ways', 'c...",United Way of Eastern Maine,https://www.charitynavigator.org/?bay=search.s...,"{'score': 81.84, 'ratingID': 138738, 'publicat...",NaN,10211478,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'ME', 'ci...","Working with the communities we serve, the Uni...",12517,"{'charityName': 'United Way of Eastern Maine',...",Live united,https://www.unitedwayem.org/
2,2,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Health', 'categoryID': 5, 'c...","{'causeID': 14, 'causeName': 'Medical Research...",The Jackson Laboratory,https://www.charitynavigator.org/?bay=search.s...,"{'score': 93.16, 'ratingID': 131608, 'publicat...","{'country': None, 'stateOrProvince': 'ME', 'ci...",10211513,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'ME', 'ci...","The Jackson Laboratory, founded in 1929, is an...",3916,"{'charityName': 'The Jackson Laboratory', 'ein...",Leading the search for tomorrow's cures,http://www.jax.org
3,3,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Arts, Culture, Humanities', ...","{'causeID': 6, 'causeName': 'Libraries, Histor...",Maine Historical Society,https://www.charitynavigator.org/?bay=search.s...,"{'score': 80.31, 'ratingID': 124121, 'publicat...",NaN,10211530,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'ME', 'ci...","Founded in 1822, the Maine Historical Society ...",15533,"{'charityName': 'Maine Historical Society', 'e...","Preserving history, engaging minds, connecting...",http://www.mainehistory.org/
4,4,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Human Services', 'categoryID...","{'causeID': 16, 'causeName': 'Youth Developmen...",Boys & Girls Clubs of Southern Maine,https://www.charitynavigator.org/?bay=search.s...,"{'score': 91.54, 'ratingID': 135033, 'publicat...",NaN,10211543,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'ME', 'ci...","For more than 100 years, Boys & Girls Clubs of...",15222,{'charityName': 'Boys & Girls Clubs of Souther...,Great Futures Start Here,http://www.bgcmaine.org/home


In [4]:
all_orgs_trim_df = all_organizations_df[['charityName', 'orgID', 'currentRating', 'mission', 'ein', 'cause']]
all_orgs_trim_df = all_orgs_trim_df.rename(columns={'charityName': 'Charity', 'orgID': 'ID_#', 'currentRating': 'Current_Rating', 'mission': 'Mission', 'ein': 'EIN_#', 'cause': 'Cause'})
all_orgs_trim_df.head()

,Charity,ID_#,Current_Rating,Mission,EIN_#,Cause
0,Mount Desert Island Biological Laboratory,5954,"{'score': 92.61, 'ratingID': 131800, 'publicat...",The MDI Biological Laboratory is a rapidly gro...,10202467,"{'causeID': 35, 'causeName': 'Non-Medical Scie..."
1,United Way of Eastern Maine,12517,"{'score': 81.84, 'ratingID': 138738, 'publicat...","Working with the communities we serve, the Uni...",10211478,"{'causeID': 42, 'causeName': 'United Ways', 'c..."
2,The Jackson Laboratory,3916,"{'score': 93.16, 'ratingID': 131608, 'publicat...","The Jackson Laboratory, founded in 1929, is an...",10211513,"{'causeID': 14, 'causeName': 'Medical Research..."
3,Maine Historical Society,15533,"{'score': 80.31, 'ratingID': 124121, 'publicat...","Founded in 1822, the Maine Historical Society ...",10211530,"{'causeID': 6, 'causeName': 'Libraries, Histor..."
4,Boys & Girls Clubs of Southern Maine,15222,"{'score': 91.54, 'ratingID': 135033, 'publicat...","For more than 100 years, Boys & Girls Clubs of...",10211543,"{'causeID': 16, 'causeName': 'Youth Developmen..."


In [5]:
#determine the data type in all columns
all_orgs_trim_df.dtypes
#Create a list of all the values stored in the Current_Rating Column
Current_Rating_Scores = all_orgs_trim_df['Current_Rating']
Current_Rating_Scores.head()

0    {'score': 92.61, 'ratingID': 131800, 'publicat...
1    {'score': 81.84, 'ratingID': 138738, 'publicat...
2    {'score': 93.16, 'ratingID': 131608, 'publicat...
3    {'score': 80.31, 'ratingID': 124121, 'publicat...
4    {'score': 91.54, 'ratingID': 135033, 'publicat...
Name: Current_Rating, dtype: object

In [8]:
#Convert the string values in the Current_Rating column into dictionaries
Current_Rating_Dictionary_Conversion = []
for values in range(len(Current_Rating_Scores)):
    ast.literal_eval(values)
    Current_Rating_Dictionary_Conversion.append()

ValueError: malformed node or string: 0

In [ ]:
#Select only the first values of each dictionary and make into a list
CRS_trim_list = [[][0] for score in (Current_Rating_Scores)]
#If data is string, trim the string to only show the number associated with 'score'
#apply a function to each of the 
